In [1]:
import ee
import geemap

In [ ]:
Map = geemap.Map()
point=ee.Geometry.Point(34.097209,47.440251)
collection = (
    ee.ImageCollection('MODIS/MCD43A4_006_NDVI').filterBounds(point)
    .filter(ee.Filter.date('2022-01-01', '2022-12-31'))   
    .select("NDVI")
)
vis_params = {
    'min': 0.0,
    'max': 1.0,
    'palette': [
        'FFFFFF',
        'CE7E45',
        'DF923D',
        'F1B555',
        'FCD163',
        '99B718',
        '74A901',
        '66A000',
        '529400',
        '3E8601',
        '207401',
        '056201',
        '004C00',
        '023B01',
        '012E01',
        '011D01',
        '011301',
    ],
}

first_image = collection.first()


# Map.addLayer(first_image, vis_params, "First image", False)
Map.setCenter(34.097209,47.440251, 8)
Map

In [3]:
def make_sequence(start, end, interval):
    date_range = end.difference(start, 'day')
    count = date_range.divide(interval).round().toInt()
    return ee.List.sequence(0, count.subtract(1)).map(
        lambda offset: start.advance(ee.Number(interval).multiply(offset), 'day'))

# Define the start and end dates and interval
start_date = ee.Date('2022-01-01')
end_date = ee.Date('2022-12-31')
interval = 10  # 10 days

# Generate a sequence of dates every 10 days
date_sequence = make_sequence(start_date, end_date, interval)

# Filter the MODIS collection based on the date sequence
modis_filtered_collection = ee.ImageCollection(ee.List([]))  # Empty collection

# Iterate over the date sequence and filter images
for date in date_sequence.getInfo():
    date = ee.Date(date['value'])
    filtered_images = collection.filter(ee.Filter.date(date, date.advance(interval, 'day'))).first()
    modis_filtered_collection = modis_filtered_collection.merge(filtered_images)

# Print the resulting image collection size
print('Image collection size:', modis_filtered_collection.size().getInfo())

Image collection size: 36


In [4]:
Map.add_time_slider(modis_filtered_collection, vis_params, time_interval=2)